In [1]:
import numpy as np
import xarray as xr
import time

In [2]:
data = xr.open_dataset("C:/Users/usuario/Desktop/TRMM-Calibration/Data/land_mask.nc")
data

<xarray.Dataset>
Dimensions:    (expver: 2, latitude: 721, longitude: 1440, time: 264)
Coordinates:
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * expver     (expver) int32 1 5
  * time       (time) datetime64[ns] 1998-01-01 1998-02-01 ... 2020-12-01
Data variables:
    lsm        (time, expver, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2021-03-02 08:42:22 GMT by grib_to_netcdf-2.16.0: /opt/ecmw...

In [3]:
data.lsm[1:3,:,1:3,1:3]

<xarray.DataArray 'lsm' (time: 2, expver: 2, latitude: 2, longitude: 2)>
array([[[[ 0.,  0.],
         [ 0.,  0.]],

        [[nan, nan],
         [nan, nan]]],


       [[[ 0.,  0.],
         [ 0.,  0.]],

        [[nan, nan],
         [nan, nan]]]], dtype=float32)
Coordinates:
  * longitude  (longitude) float32 0.25 0.5
  * latitude   (latitude) float32 89.75 89.5
  * expver     (expver) int32 1 5
  * time       (time) datetime64[ns] 1998-02-01 1998-03-01
Attributes:
    units:          (0 - 1)
    long_name:      Land-sea mask
    standard_name:  land_binary_mask

In [4]:
np.nansum(data.lsm[1:3,:,1:3,1:3], axis = 1)

array([[[0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.]]], dtype=float32)

In [5]:
data.lsm[:,:,1:3,1:3].shape, np.nansum(data.lsm[:,:,1:3,1:3], axis = 1).shape

((264, 2, 2, 2), (264, 2, 2))

In [6]:
start_time = time.time()
lsm_modified = np.nansum(data.lsm, axis = 1)
print("--- %s seconds ---" % (time.time() - start_time))

--- 57.93597650527954 seconds ---


In [7]:
ds = xr.Dataset({
    "lsm": (["time", "latitude", "longitude"], lsm_modified)
    },
    coords = {
        "longitude": (["longitude"], data.longitude),
        "latitude": (["latitude"], data.latitude),
        "time": (["time"], data.time)
    }
    )

In [8]:
ds

<xarray.Dataset>
Dimensions:    (latitude: 721, longitude: 1440, time: 264)
Coordinates:
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 1998-01-01 1998-02-01 ... 2020-12-01
Data variables:
    lsm        (time, latitude, longitude) float32 0.0 0.0 0.0 ... 1.0 1.0 1.0

In [9]:
ds.attrs["Conventions"] = data.Conventions
ds.attrs["history"] = data.history
ds

<xarray.Dataset>
Dimensions:    (latitude: 721, longitude: 1440, time: 264)
Coordinates:
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 1998-01-01 1998-02-01 ... 2020-12-01
Data variables:
    lsm        (time, latitude, longitude) float32 0.0 0.0 0.0 ... 1.0 1.0 1.0
Attributes:
    Conventions:  CF-1.6
    history:      2021-03-02 08:42:22 GMT by grib_to_netcdf-2.16.0: /opt/ecmw...

In [10]:
ds.longitude.attrs = data.longitude.attrs
ds.latitude.attrs = data.latitude.attrs
ds.time.attrs = data.time.attrs
ds.lsm.attrs = data.lsm.attrs

ds

<xarray.Dataset>
Dimensions:    (latitude: 721, longitude: 1440, time: 264)
Coordinates:
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 1998-01-01 1998-02-01 ... 2020-12-01
Data variables:
    lsm        (time, latitude, longitude) float32 0.0 0.0 0.0 ... 1.0 1.0 1.0
Attributes:
    Conventions:  CF-1.6
    history:      2021-03-02 08:42:22 GMT by grib_to_netcdf-2.16.0: /opt/ecmw...

In [11]:
ds.to_netcdf("C:/Users/usuario/Desktop/TRMM-Calibration/Data/land_mask_mod.nc")